In [32]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge
from collections import Counter

In [2]:
pd.set_option('display.max_columns', 100)

# Data Prep

In [3]:
manifest_df = pd.read_csv('../Manifest/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [4]:
from clean import assay_transpose

In [5]:
assay_df = pd.read_csv('../TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv')
assay_t_df = assay_transpose(assay_df)

In [6]:
from clean import assay_clinical_merge

In [7]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

# Modeling on AML_genes

## Classification - predict low_risk / not_low_risk in AML patients

1) Variance comparison: Subset of patients under 03 09A classifcation - choosing 1000 genes of highest/lowest variance 
between labeled low_risk, not_low_risk group

2) Model performance when randomly sampling genes

3) Lasso logistic regression

4) Tree modeling with features considered important by Lasso

In [46]:
from model_comp import data_prep_columns, model_prep, model_comp, model_prep_loc, xgboost_tuner, create_model_df
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# 1. Variance Comparison

### High Variance

In [23]:
df, a = data_prep_columns(AML_genes, 'Max')
X_train, X_test, y_train, y_test, holdout, df = model_prep(df, a)
xg1, rf1 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.28771180389607903
Gradient Boost Log Loss 0.36210740308746076
Random Forest Log Loss 0.3262005795045274
XGBoost Log Loss 0.22184269636164647


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [25]:
df.head()

,ENSG00000129824.14,ENSG00000079102.15,ENSG00000229807.8,ENSG00000152192.7,ENSG00000012817.14,ENSG00000067048.15,ENSG00000131002.10,ENSG00000198692.8,ENSG00000183878.14,ENSG00000185686.16,ENSG00000233864.6,ENSG00000067646.10,ENSG00000133454.14,ENSG00000226278.1,ENSG00000134184.11,ENSG00000114374.11,ENSG00000130508.9,ENSG00000099725.13,ENSG00000169855.18,ENSG00000260197.1,ENSG00000143995.18,ENSG00000228651.1,ENSG00000158481.11,ENSG00000260918.1,ENSG00000107562.15,ENSG00000124780.12,ENSG00000145358.5,ENSG00000198848.11,ENSG00000100079.6,ENSG00000169908.9,ENSG00000078399.14,ENSG00000132514.12,ENSG00000251381.5,ENSG00000227502.2,ENSG00000233392.4,ENSG00000156575.2,ENSG00000231513.3,ENSG00000230882.1,ENSG00000205502.3,ENSG00000152785.6,ENSG00000128422.14,ENSG00000007038.9,ENSG00000102854.13,ENSG00000150760.11,ENSG00000267453.5,ENSG00000170549.3,ENSG00000125869.8,ENSG00000267206.4,ENSG00000164176.11,ENSG00000204644.8,...,ENSG00000242193.8,ENSG00000196209.11,ENSG00000176105.12,ENSG00000078596.9,ENSG00000234456.6,ENSG00000163600.11,ENSG00000165349.10,ENSG00000272168.4,ENSG00000158186.11,ENSG00000235478.3,ENSG00000167641.9,ENSG00000133863.5,ENSG00000165731.16,ENSG00000163331.9,ENSG00000119121.20,ENSG00000166928.9,ENSG00000235098.7,ENSG00000163735.6,ENSG00000169427.5,ENSG00000147614.3,ENSG00000264660.2,ENSG00000206588.1,ENSG00000172889.14,ENSG00000112303.12,ENSG00000134121.8,ENSG00000213853.8,ENSG00000171016.10,ENSG00000184304.13,ENSG00000167748.9,ENSG00000145864.11,ENSG00000211752.3,ENSG00000182632.13,ENSG00000207205.1,ENSG00000106031.7,ENSG00000157510.12,ENSG00000145777.13,ENSG00000183850.12,ENSG00000105366.14,ENSG00000169851.14,ENSG00000267886.1,ENSG00000101115.11,ENSG00000102230.12,ENSG00000101307.14,ENSG00000188282.11,ENSG00000134901.11,ENSG00000179776.16,ENSG00000134028.13,ENSG00000120332.14,ENSG00000238243.3,ENSG00000267568.5
5,-5.80459,-4.74701,7.15795,-3.1249,-6.92715,-5.18099,-4.74701,-6.92714,-6.92714,-3.39161,-4.74701,-6.92715,-6.92715,1.99433,-2.24212,-5.80459,-0.565464,-5.18099,-1.94482,-6.92714,5.32477,0.0342299,0.383566,-1.74447,5.68118,-1.3045,0.0745502,6.44677,4.56271,-1.3394,5.56953,3.85028,-1.37515,-0.390612,-4.74701,-2.11576,-6.92714,-0.740952,-3.54606,-5.80459,-2.61702,-5.18099,8.26967,-3.71906,-6.92714,-6.92715,9.2241,-6.92714,-5.18099,-6.92714,...,2.3578,5.77491,0.126615,-1.44944,-3.54606,-2.53384,-6.92715,3.00333,4.02848,2.75306,-1.17281,-1.17281,6.42192,-0.504794,-6.92715,3.45665,-4.14335,-3.71906,-3.71906,-3.1249,-6.92714,2.85303,0.782873,-2.53384,-1.52776,-0.88817,-3.71906,-3.91566,6.43084,-2.61702,-4.74701,-5.80459,2.53682,-2.45519,-1.89207,-4.41384,-1.02348,-1.99958,-5.80459,-0.284807,-0.88817,-2.89989,7.51086,0.496696,-2.79931,0.225427,-1.84117,0.113774,-3.91566,-6.92714
9,-1.76765,-4.50961,8.21935,-3.23314,-2.4217,-0.028267,-3.23314,-2.49236,-2.72784,-2.72784,-4.24272,-2.28999,-3.35954,0.898223,4.88921,-2.90931,-4.83733,-4.01759,-5.86665,-4.24272,4.955,1.35088,-4.83733,-5.86665,-2.81572,1.35592,-4.83733,2.96277,3.36624,1.45313,6.74335,-2.05793,-5.86665,2.38843,-0.0414518,-4.01759,-6.92714,-4.01759,-5.26208,-4.50961,2.6217,-2.56667,-3.23314,4.88747,-3.65137,-6.92715,-2.22838,-4.24272,-1.68256,-6.92714,...,2.92768,1.80214,4.88443,6.5013,0.0973502,-0.787976,-6.92715,-2.4217,-0.660448,-5.26208,-3.35954,-5.86665,-0.209853,-0.856232,-6.92715,1.33056,-3.35954,-3.82289,-5.26207,0.898223,-6.92714,-5.86665,4.51667,1.39069,-6.92715,-2.22838,-2.28999,-5.26208,-2.4217,-5.86665,1.92585,-6.92714,-4.24272,-5.26207,-2.11254,-1.81215,0.267381,-4.83733,-5.86665,-0.452428,-2.645,-2.56667,3.04615,-2.72784,2.8188,-3.35954,-0.318004,0.972251,1.10383,-6.92714
14,7.58226,-5.93369,-3.3542,-3.61702,5.45232,8.17851,5.57421,4.17348,5.11027,0.787718,4.80919,5.97543,-4.35311,3.01975,4.04274,5.31742,0.239735,4.93395,-0.185255,1.87675,6.44452,2.25205,1.5151,-0.940059,-3.3542,-5.35092,-2.29588,5.43206,4.78681,-4.35311,7.78952,-3.23881,-4.35311,2.31715,1.61819,-2.29588,-1.32529,-2.76961,-2.85203,-1.13267,-5.93369,5.4402,9.59395,0.780897,-0

In [29]:
xg1_important = df.columns[np.array(xg1.feature_importances_) > 0]
rf1_important = df.columns[np.array(rf1.feature_importances_) > 0]

In [30]:
print(len(xg1_important))
len(rf1_important)

205


697

### country frequency of genes being labeled 'important' by RF or XGboost

In [35]:
xg_gene_counter = Counter()
rf_gene_counter = Counter()

In [36]:
for gene in xg1_important:
    xg_gene_counter[gene] +=1
for gene in rf1_important:
    rf_gene_counter[gene] +=1

### 1b. low variance genes between two labeled groups

In [38]:
df, a = data_prep_columns(AML_genes, 'Min')
X_train, X_test, y_train, y_test, holdout, df = model_prep(df, a)
xg2, rf2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['label'] = y.copy()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4236865689681798
Gradient Boost Log Loss 0.49256451159202913
Random Forest Log Loss 0.48223471620602953
XGBoost Log Loss 0.3429010650394736


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [39]:
xg2_important = df.columns[np.array(xg2.feature_importances_) > 0]
rf2_important = df.columns[np.array(rf2.feature_importances_) > 0]

In [40]:
for gene in xg2_important:
    xg_gene_counter[gene] +=1
for gene in rf2_important:
    rf_gene_counter[gene] +=1

# 2.random gene selection

- random_cols_ are **column index numbers** chosen at random

- does not perform as well as the variation selected columns

- use **model_prep** instead of **model_prep_loc** as random_columns_ is a randomly generated list of numbers 
used to select columns

In [53]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]

In [57]:
# random_cols_ are column index numbers

In [54]:
len(random_cols_)

1912

In [64]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, df = model_prep(df, random_cols_)
xg_random, rf_random = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  holdout = data.sample(frac=.2, random_state=8)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3516842391800261
Gradient Boost Log Loss 0.42697625609573914
Random Forest Log Loss 0.41377105849406226
XGBoost Log Loss 0.29220552223961094


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [65]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, df = model_prep(df, random_cols_)
xg_random2, rf_random2 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  holdout = data.sample(frac=.2, random_state=8)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.4030763054992051
Gradient Boost Log Loss 0.45468446485353564
Random Forest Log Loss 0.40352478878670006
XGBoost Log Loss 0.5104591532333477


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [66]:
random_cols = np.random.choice(21404, 2000)
random_set = set(random_cols)
random_cols = list(random_set)
random_cols_ = [x + 84 for x in random_cols]
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout, df = model_prep(df, random_cols_)
xg_random3, rf_rando3 = model_comp(X_train, X_test, y_train, y_test)

/Users/david/statsbomb/RNAseq_Cancer_Biomarkers/scripts/model_comp.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  holdout = data.sample(frac=.2, random_state=8)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.3878542167142077
Gradient Boost Log Loss 0.4848117568424528
Random Forest Log Loss 0.41450316331528253
XGBoost Log Loss 0.3318681287000308


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


# 3. Lasso Logistic Regression using all columns

- lasso reduces to 3212 genes instead of 20,000+

In [69]:
from sklearn.linear_model import LogisticRegression

In [70]:
lasso_columns = df.iloc[:, 84:-4].columns
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, holdout = model_prep_loc(df, lasso_columns)
log_model = LogisticRegression(penalty='l1', solver='saga')
log_model.fit(X_train, y_train)
y_pred = log_model.predict_proba(X_test)
log_loss(y_test, y_pred)

/Users/david/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.21266221675736335

In [73]:
log_coefs = np.array(log_model.coef_)
mask = log_coefs > 0
important_lasso_genes = lasso_columns[mask[0]]

In [74]:
len(important_lasso_genes)

3212

In [ ]:
# for gene in important_lasso_genes:
#     xg_gene_counter[gene] +=1
#     rf_gene_counter[gene] +=1

### 4. Trees Performance Using Lasso Genes

In [81]:
df, __ = data_prep_columns(AML_genes, 'neither')
X_train, X_test, y_train, y_test, X = model_prep_loc(df, important_lasso_genes)
xg_lass, rf_lass = model_comp(X_train, X_test, y_train, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Ensemble Log Loss 0.38184477879777423
Gradient Boost Log Loss 0.43236141770308356
Random Forest Log Loss 0.3916117319651301
XGBoost Log Loss 0.38323753604010957


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [82]:
xg_lass_important = X.columns[np.array(xg_lass.feature_importances_) > 0]
rf_lass_important = X.columns[np.array(rf_lass.feature_importances_) > 0]

In [83]:
for gene in xg_lass_important:
    xg_gene_counter[gene] +=1
for gene in rf_lass_important:
    rf_gene_counter[gene] +=1